# COVID-19 analysis
> Analyzing coronavirus total cases, deaths and new cases by country.

- comments: true
- author: Eric Wulff
- categories: [overview, interactive]
- hide: true

In [ ]:
#hide
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from datetime import datetime
import plotly.graph_objects as go

from ipywidgets import widgets


from utils import datetimeify, process_df

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_covid19_{name}_global.csv')
    return pd.read_csv(url, index_col='Country/Region')

width = 2

# function definitions
def total_vs_time(df, descr):
    date_list = datetimeify(df.index)

    traces = []
    # Add traces, one for each slider step
    for ii, country in enumerate(df.keys()):
        if country == 'Sweden':
            width = 4
        else:
            width = 2
        traces.append(
            dict(
                line=dict(width=width),
                mode='lines+markers',
                name=str(country),
                x=date_list,
                y=df[country]))

    layout = dict(
        title='Covid-19 {}'.format(descr),
        autosize=True,
        width=900,
        height=600,
    )

    return traces, layout


def new_vs_total(df, descr, window=1):
    # Create figure
    traces = []

    # Add traces, one for each slider step
    for ii, country in enumerate(df.keys()):
        if country == 'Sweden':
            width = 4
        else:
            width = 2
        traces.append(
            dict(
                line=dict(width=width),
                name=str(country),
                mode='lines+markers',
                x=df[country].rolling(window=window).mean(),
                y=df.diff()[country].rolling(window=window).mean()))

    layout = dict(title='Covid-19 {} rolling mean of {} days'.format(descr, window),
                  autosize=True,
                  width=900,
                  height=600,
                  )
    return traces, layout


def new_vs_time(df, descr, window=1, countries=['Sweden', 'Norway', 'Denmark', 'Finland']):
    date_list = datetimeify(df.index)

    # Create figure
    traces = []
    # Add traces, one for each slider step
    for ii, country in enumerate(countries):
        traces.append(
            dict(
                line=dict(width=width),
                mode='lines+markers',
                name=str(country),
                x=date_list[39:],
                y=df.iloc[39:].diff()[country].rolling(window).mean()))

    layout = dict(
        title='Covid-19 new {} rolling mean of {} days'.format(descr, window),
        autosize=True,
        width=900,
        height=600,
    )

    return traces, layout

In [ ]:
#hide
df_conf = process_df(get_frame('confirmed'))
df_deaths = process_df(get_frame('deaths'))

In [ ]:
#hide
# Pick countries to plot
countries = ['China',
             'Sweden',
             'Denmark',
             'Norway',
             'France',
             'Spain',
             'Germany',
             'Switzerland',
             'Finland',
             'US',
             'Korea, South',
             'Singapore',
             'Italy',
             ]
countries.sort()

# Filter DataFrames
df_conf = df_conf[countries]
df_deaths = df_deaths[countries]

In [ ]:
#hide-input
fig = go.Figure(layout=layout_conf)

traces_conf, layout_conf = total_vs_time(df_conf, descr='confirmed cases')
traces_deaths, layout_deaths = total_vs_time(df_deaths, descr='deaths')

for trace in traces_deaths:
    trace.update({'visible': False})

fig.add_traces(traces_conf)
fig.add_traces(traces_deaths)

fig.update_layout(yaxis_type="log",
                  xaxis_title='Date',
                  yaxis_title='Covid-19 {}'.format(descr),
                  title='Covid-19 {}'.format(descr))

visible_list = [True for ii in range(len(traces_conf))] + [False for ii in range(len(traces_deaths))]

fig.update_layout(
    updatemenus=[
        dict(
            x=0,
            xanchor='left',
            y=1.08,
            active=0,
            buttons=list([
                dict(label="Confirmed cases",
                     method="update",
                     args=[{"visible": visible_list},
                          ]),
                dict(label="Confirmed deaths",
                     method="update",
                     args=[{"visible": np.invert(visible_list)},
                ]),
            ]),
        ),
                dict(
            x=0.22,
            xanchor='left',
            y=1.08,
            active=1,
            buttons=list([
            dict(label="Linear",
                 method="relayout",
                 args=["yaxis.type", '']),
            dict(label="Logarithmic",
                 method="relayout",
                 args=["yaxis.type", 'log']),
        ]),
        )
    ])

fig.layout.template = 'plotly_white+xgridoff'
fig.show()

In [ ]:
#hide
import plotly
plotly.io.templates

In [ ]:
#hide-input
window = 7
fig = go.Figure(layout=layout_conf)

traces_conf, layout_conf = new_vs_time(df_conf, descr='confirmed cases', window=window)
traces_deaths, layout_deaths = new_vs_time(df_deaths, descr='deaths', window=window)

for trace in traces_deaths:
    trace.update({'visible': False})

fig.add_traces(traces_conf)
fig.add_traces(traces_deaths)

fig.update_layout(
                  yaxis_title='New {} per day'.format(descr),
                  xaxis_title='Total {}'.format(descr),
                  title='Covid-19 {} rolling mean of {} days'.format(descr, window),
                 )

fig.update_layout(
    updatemenus=[
        dict(
            x=0,
            xanchor='left',
            y=1.08,
            active=0,
            buttons=list([
                dict(label="Confirmed cases",
                     method="update",
                     args=[{"visible": [True, True, True, True, False, False, False, False]},
                          ]),
                dict(label="Confirmed deaths",
                     method="update",
                     args=[{"visible": [False, False , False , False, True, True, True, True]},
                ]),
            ]),
        ),
                dict(
            x=0.22,
            xanchor='left',
            y=1.08,
            active=0,
            buttons=list([
            dict(label="Linear",
                 method="relayout",
                 args=["yaxis.type", '']),
            dict(label="Logarithmic",
                 method="relayout",
                 args=["yaxis.type", 'log']),
        ]),
        )
    ])

fig.layout.template = 'plotly_white+xgridoff'
fig.show()